In [1]:
#-*- coding: utf-8 -*-

import yaml
import os
from pyspark import SparkContext, SparkConf
import numpy as np
import sys
import csv

import findspark

import ankus4passDALib.unsupervised.kmeans_pp_cluster as kmeans_pp_cluster

bath_path = os.path.dirname(os.path.abspath('__file__'))
print bath_path
yaml_file = open(bath_path + '/../etc/spark_config_local.yaml', 'r')

spark_yaml = yaml.load(yaml_file)
install_path = spark_yaml["spark_env"]
print install_path
SPARK_LOCAL_IP = spark_yaml["SPARK_LOCAL_IP"]
PYSPARK_DRIVER_PYTHON = spark_yaml["PYSPARK_DRIVER_PYTHON"]
PYSPARK_DRIVER_PYTHON_OPTS = spark_yaml["PYSPARK_DRIVER_PYTHON_OPTS"]

os.environ["SPARK_HOME"] = install_path
os.environ["SPARK_LOCAL_IP"] = SPARK_LOCAL_IP
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_DRIVER_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = PYSPARK_DRIVER_PYTHON_OPTS
os.environ["PYSPARK_PYTHON"] = 'python2'
os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"]='YES' # for mac fork error
findspark.init()
K = 10
numbers_of_parallel_kmeans = 20
converge_dist = 0.03
bath_path = os.path.dirname(os.path.abspath('__file__'))
input_path = bath_path + '/../DataSet/iris.csv'
output_path = '../model/iris.pkl'
print 'Run with default argument'
print 'K', K, 'Parallel Kmeans', numbers_of_parallel_kmeans, ' Converge Dist', converge_dist
print 'source data', input_path
seed = 1000

vector_list = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
object_name = ['class']

input_raw_data_list = []

with open(input_path, "r") as f:
    mycsv = csv.DictReader(f)
    for row in mycsv:
        vector_values = []
        for col in vector_list:
            vector_values.extend([row[col]])
        vector_values_np = np.array(list(map(float, vector_values)))
        for col in object_name:
            input_raw_data_list.append((row[col], vector_values_np))

conf = SparkConf()
conf.setMaster('local')
conf.setAppName('kmeans')
conf.set("spark.executor.memory", "6800M")
conf.set('spark.executor.cores', 1)
sc = SparkContext(conf = conf).getOrCreate()
rdd = sc.parallelize(input_raw_data_list)  # 'str type object name, numpy vector'
print rdd.take(3)
print 'K', K
print 'RUNS', numbers_of_parallel_kmeans
np.random.seed(seed)
kmeans_pp = kmeans_pp_cluster.kmeans_pp()
kmeans_pp.fit(rdd, K, numbers_of_parallel_kmeans, converge_dist, np.random.randint(1000))
best_model = kmeans_pp.save_best_model(output_path, rdd, kmeans_pp.centroids)

group2 = rdd.map(lambda (in_vector): (in_vector[0], 'cluster:' + str(kmeans_pp.get_closest_centers(in_vector[1], best_model)))).collect()
print 'cluster assign'
for element in group2:
    print element
sc.stop()


/Users/prismdata/Documents/ankus_analyzer_p_workspace/Bigdata_ML_Library/ankus4pass_DLib/ankus4passDALib/example
/Users/prismdata/spark-2.3.2-bin-hadoop2.6
Arguments is not valid
Run with default argument
K 10 Parallel Kmeans 20  Converge Dist 0.03
source data /Users/prismdata/Documents/ankus_analyzer_p_workspace/Bigdata_ML_Library/ankus4pass_DLib/ankus4passDALib/example/../DataSet/iris.csv


/Users/prismdata/Documents/ankus_analyzer_p_workspace/ankus_analyzer_P/ankus4paas/Bigdata_ML_Library/ankus4pass_env_py2/lib/python2.7/site-packages/ipykernel_launcher.py:18: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


[('Iris-setosa', array([5.1, 3.5, 1.4, 0.2])), ('Iris-setosa', array([4.9, 3. , 1.4, 0.2])), ('Iris-setosa', array([4.7, 3.2, 1.3, 0.2]))]
K 10
RUNS 20
5.266659325990902 4.697233419965684 4.852439200202272
cluster assign
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster:1')
('Iris-setosa', 'cluster: